In [1]:
from STvAR_v3 import *
from STvARMetricObjects import *
from STvARFiniteDifference import *
from STvARAdvection import *
from STvARSymbolParsing import *
from STvARIndexing import*

In [2]:
p0r = stvar('Real_0', gridvar = True, vartype = 'p_fab', isreal=True)
p0i = stvar('Imaginary_0', gridvar = True, vartype = 'p_fab', isreal=True)

p1r = stvar('Real_1', gridvar = True, vartype = 'p_fab', isreal=True)
p1i = stvar('Imaginary_1', gridvar = True, vartype = 'p_fab', isreal=True)

p_U = stvarrank1('p_U', gridvar = True, vartype = 'U_fab', dim = 2)
p_U.isymb[0] = p0r.isymb + sp.I*p0i.isymb
p_U.isymb[1] = p1r.isymb + sp.I*p1i.isymb

u0r = stvar('Real_0', gridvar = True, vartype = 'U_fab', isreal=True)
u0i = stvar('Imaginary_0', gridvar = True, vartype = 'U_fab', isreal=True)

u1r = stvar('Real_1', gridvar = True, vartype = 'U_fab', isreal=True)
u1i = stvar('Imaginary_1', gridvar = True, vartype = 'U_fab', isreal=True)

u_U = stvarrank1('u_U', gridvar = True, vartype = 'U_fab', dim = 2)
u_U.isymb[0] = u0r.isymb + sp.I*u0i.isymb
u_U.isymb[1] = u1r.isymb + sp.I*u1i.isymb

In [3]:
gamma_1 = sp.Matrix([[0,1],[1,0]])
gamma_2 = sp.Matrix([[0,-sp.I],[sp.I,0]])
gamma_3 = sp.Matrix([[1,0],[0,-1]])

In [4]:
def sigma1Psi(Psi_U):
    sigma1Psi_U = stvarrank1('sigma1Psi_U', dim = 2)
    
    sigma1Psi_U_tmp = gamma_1*sp.Matrix([Psi_U.isymb[0], Psi_U.isymb[1]])
    
    sigma1Psi_U.isymb[0] = sigma1Psi_U_tmp[0]
    sigma1Psi_U.isymb[1] = sigma1Psi_U_tmp[1]
    
    return sigma1Psi_U

def sigma2Psi(Psi_U):
    sigma2Psi_U = stvarrank1('sigma2Psi_U', dim = 2)
    
    sigma2Psi_U_tmp = gamma_2*sp.Matrix([Psi_U.isymb[0], Psi_U.isymb[1]])
    
    sigma2Psi_U.isymb[0] = sigma2Psi_U_tmp[0]
    sigma2Psi_U.isymb[1] = sigma2Psi_U_tmp[1]
    
    return sigma2Psi_U

def sigma3Psi(Psi_U):
    sigma3Psi_U = stvarrank1('sigma3Psi_U', dim = 2)
    
    sigma3Psi_U_tmp = gamma_3*sp.Matrix([Psi_U.isymb[0], Psi_U.isymb[1]])
    
    sigma3Psi_U.isymb[0] = sigma3Psi_U_tmp[0]
    sigma3Psi_U.isymb[1] = sigma3Psi_U_tmp[1]
    
    return sigma3Psi_U

In [5]:
def MuDPsi(psi_U,u_U, r):
    MuDpsi_U = stvarrank1('uDpsi_U', dim = 2)
    
    conjugateu_U = stvarrank1('conju_U', dim = 2)
    conjugateu_U.isymb[0] = sp.conjugate(u_U.isymb[0])
    conjugateu_U.isymb[1] = sp.conjugate(u_U.isymb[1])
    
    psi_U_tmp = sp.Matrix([psi_U.isymb[0], psi_U.isymb[1]])
    
    M1 = sp.eye(2) - gamma_1
    M2 = sp.eye(2) - gamma_2
    
    Mm1 = sp.eye(2) + gamma_1
    Mm2 = sp.eye(2) + gamma_2
    
    M1psi_U = M1*psi_U_tmp
    M2psi_U = M2*psi_U_tmp
    
    Mm1psi_U = Mm1*psi_U_tmp
    Mm2psi_U = Mm2*psi_U_tmp
    
    MuDpsi_U.isymb[0] = u_U.isymb[0]*shift(M1psi_U[0],[1,0,0,0])+u_U.isymb[1]*shift(M2psi_U[0],[0,1,0,0])
    MuDpsi_U.isymb[1] = u_U.isymb[0]*shift(M1psi_U[1],[1,0,0,0])+u_U.isymb[1]*shift(M2psi_U[1],[0,1,0,0])
    
    MuDpsi_U.isymb[0] += shift(conjugateu_U.isymb[0],[-1,0,0,0])*shift(Mm1psi_U[0],[-1,0,0,0])+shift(conjugateu_U.isymb[1],[0,-1,0,0])*shift(Mm2psi_U[0],[0,-1,0,0])
    MuDpsi_U.isymb[1] += shift(conjugateu_U.isymb[0],[-1,0,0,0])*shift(Mm1psi_U[1],[-1,0,0,0])+shift(conjugateu_U.isymb[1],[0,-1,0,0])*shift(Mm2psi_U[1],[0,-1,0,0])
    
    return MuDpsi_U

In [6]:
r = sp.symbols('r', real = True)
m = sp.symbols('m_0', real = True)

In [7]:
def OpPsi(Psi_U, u_U, m, r):
    OpPsi_U = stvarrank1('OpPsi_U', dim = 2)
    
    OpPsi_U.isymb[0] = (m+2)*Psi_U.isymb[0]
    OpPsi_U.isymb[1] = (m+2)*Psi_U.isymb[1]
    
    MuDPsi_U = MuDPsi(Psi_U, u_U, r)
    
    OpPsi_U.isymb[0] += MuDPsi_U.isymb[0]
    OpPsi_U.isymb[1] += MuDPsi_U.isymb[1]
    
    OpPsi_U.isymb[0] = sp.simplify(OpPsi_U.isymb[0])
    OpPsi_U.isymb[1] = sp.simplify(OpPsi_U.isymb[1])
    
    
    return OpPsi_U

In [8]:
OpPsi_U = sigma3Psi(OpPsi(p_U, u_U, m, r))

In [9]:
OpPsiReal_U = stvarrank1('OpPsiReal_U', isreal = True, dim = 2)
OpPsiReal_U.isymb[0] = sp.re(OpPsi_U.isymb[0])
OpPsiReal_U.isymb[1] = sp.re(OpPsi_U.isymb[1])

OpPsiImaginary_U = stvarrank1('OpPsiImaginary_U', isreal = True, dim = 2)
OpPsiImaginary_U.isymb[0] = sp.im(OpPsi_U.isymb[0])
OpPsiImaginary_U.isymb[1] = sp.im(OpPsi_U.isymb[1])

In [11]:
print(OpPsiReal_U.symb2isymb().replace("p_fabIdx","cIdx").replace('U_fabIdx','cIdx'))
print(OpPsiImaginary_U.symb2isymb().replace("p_fabIdx","cIdx").replace('U_fabIdx','cIdx'))

        amrex::Real OpPsiReal_U_0 = (m_0 + 2)*p_fab(i, j, k, cIdx::Real_0) + U_fab(i - 1, j, k, cIdx::Imaginary_0)*p_fab(i - 1, j, k, cIdx::Imaginary_0) + U_fab(i - 1, j, k, cIdx::Imaginary_0)*p_fab(i - 1, j, k, cIdx::Imaginary_1) + U_fab(i - 1, j, k, cIdx::Real_0)*p_fab(i - 1, j, k, cIdx::Real_0) + U_fab(i - 1, j, k, cIdx::Real_0)*p_fab(i - 1, j, k, cIdx::Real_1) + U_fab(i, j - 1, k, cIdx::Imaginary_1)*p_fab(i, j - 1, k, cIdx::Imaginary_0) - U_fab(i, j - 1, k, cIdx::Imaginary_1)*p_fab(i, j - 1, k, cIdx::Real_1) + U_fab(i, j - 1, k, cIdx::Real_1)*p_fab(i, j - 1, k, cIdx::Imaginary_1) + U_fab(i, j - 1, k, cIdx::Real_1)*p_fab(i, j - 1, k, cIdx::Real_0) - U_fab(i, j, k, cIdx::Imaginary_0)*p_fab(i + 1, j, k, cIdx::Imaginary_0) + U_fab(i, j, k, cIdx::Imaginary_0)*p_fab(i + 1, j, k, cIdx::Imaginary_1) - U_fab(i, j, k, cIdx::Imaginary_1)*p_fab(i, j + 1, k, cIdx::Imaginary_0) - U_fab(i, j, k, cIdx::Imaginary_1)*p_fab(i, j + 1, k, cIdx::Real_1) + U_fab(i, j, k, cIdx::Real_0)*p_fab(i + 1, j, k, 